# SAé 2.04 exploitations d'une base de données

#Introduction

Dans ce document, nous allons chercher à analyser les données concernant les étudiants et leurs stages effectués en 2ème année de DUT informatique.

L'objectif est d'en faire un rapport détaillé avec des graphes pour visualiser les pistes d'amélioration potentielles concernant les stages des étudiants, et également pour améliorer les systèmes éducatifs.

##Table des matières
1. Importation des librairies et des données
2. Nettoyage des données
3. Création de nouvelles variables
4. Visualisation et analyse des données
  1. Nombre d'étudiants dans chaque domaine en stage
  2. Appréciation
  3. Nombre d'étudiants en stage chaque année
  4. Temps de stage
  5. Moyenne_Ecart-type_Variance
  6. Moyenne en fonction du diplôme
  7. Evolution des moyennes entre le s3 et s4
  8. Absences et notes
5. Tableau des données analysées

# Importation des librairies et des données

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv', delimiter=';')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

# Nettoyage des données

In [ ]:
# Filtrer les lignes où les dates de début ou de fin contiennent un tiret
date = df[~(df['debut_stage'].str.contains('-') | df['fin_stage'].str.contains('-'))]

# Convertir les colonnes de dates en objets datetime
date['debut_stage'] = pd.to_datetime(date['debut_stage'], format='%d/%m/%Y')
date['fin_stage'] = pd.to_datetime(date['fin_stage'], format='%d/%m/%Y')


# Remplacer les virgules par des points dans les colonnes contenant des notes
df['note_s3'] = df['note_s3'].str.replace(',', '.')
df['note_s4'] = df['note_s4'].str.replace(',', '.')

# Convertir tous les mots en minuscules dans la colonne description_sujet
df['type_stage'] = df['type_stage'].str.lower()

# Remplacer les occurrences de "STI2D" par "STMG" dans la colonne bac pour les analyser ensemble
df['bac'] = df['bac'].replace('STI2D', 'Techno')
df['bac'] = df['bac'].replace('STMG', 'Techno')

# Convertir les colonnes en type numérique
colonnes_notes_semestres = ['note_s3','note_s4']
colonnes_notes_stages = ['rapport_fond', 'rapport_forme', 'soutenance_fond', 'soutenance_forme','soutenance_echange_jury', 'mission_methode', 'mission_travail_fourni', 'mission_comportement']
df[colonnes_notes_stages] = df[colonnes_notes_stages].apply(pd.to_numeric, errors='coerce')
df[colonnes_notes_semestres] = df[colonnes_notes_semestres].apply(pd.to_numeric, errors='coerce')
df.head()

# Création de nouvelles variables

Calculer la moyenne des notes de l'année

In [ ]:
df['moyenne_Notes_Semestre'] = df[colonnes_notes_semestres].mean(axis=1)

Calculer la moyenne des notes du stage

In [ ]:
# Coefficients correspondants aux notes des stages
coefficients_stages = [30, 10, 15, 7.5, 7.5, 10, 10, 10]

# Calculer la moyenne pondérée des colonnes spécifiées pour les stages
df['moyenne_Notes_Stage'] = (df[colonnes_notes_stages] * coefficients_stages).sum(axis=1) / sum(coefficients_stages)

Calculer la moyenne totale de l'élève sur l'année et les classés avec des appréciations

In [ ]:
# Calculer la moyenne des deux autres colonnes
df['moyenne'] = df[['moyenne_Notes_Semestre', 'moyenne_Notes_Stage']].mean(axis=1)

# Définir les intervalles et les étiquettes pour chaque appréciation
intervals = [0, 8, 12, 14, 16, 18, 20]
labels = ['Médiocre', 'Insuffisant', 'Suffisant', 'Bien', 'Très bien', 'Excellent']

# Créer une nouvelle colonne "appréciation" en utilisant pd.cut()
df['appréciation'] = pd.cut(df['moyenne'], bins=intervals, labels=labels, right=False)

# Visualisation et analyse des données

##Nombre d'éudiants dans chaque domaine en stage

**Description**

Tout d'abord, nous allons observer le nombre d'occurrences d'élèves dans chaque domaine durant leur stage entre 2017 et 2022.

In [ ]:
# Compter le nombre d'occurrences de chaque mot
word_counts = df['type_stage'].value_counts()

# Afficher les résultats sous forme de diagramme à barres
word_counts.plot(kind='bar', color='turquoise', title="Nombre d'éudiants dans chaque domaine en stage entre 2017 et 2022",xlabel='Domaines',ylabel='Occurences',figsize=(12, 6))

**Analyse :**

Ce graphique nous permet d'observer que beaucoup d'étudiants effectuent leurs stages dans le domaine du développement.

Cela peut-il s'expliquer par un intérêt commun pour ce domaine ou par un nombre d'offres plus important ?

Pour répondre à cette question, nous avons recherché sur deux sites d'annonces d'emplois (LinkedIn et Indeed) le nombre d'annonces que nous pouvions trouver pour les deux domaines. Sur LinkedIn, les résultats sont serrés : 564 annonces pour le développement contre 511 pour un autre domaine. En revanche, sur Indeed, on observe plus de 2000 offres en développement web contre 700 en développement.

Une conclusion rapide possible est donc que le développement attire plus les stagiaires de BUT Informatique malgré une forte demande en web.



---



##Appréciation

**Description**

Ici, nous cherchons à observer le nombre d'élèves dans chaque mention, ces mentions ayant été créées plus haut dans le document.

In [ ]:
# Compter le nombre d'occurrences de chaque mot
appreciationCount = df['appréciation'].value_counts()

# Afficher les résultats sous forme de diagramme à barres
appreciationCount.plot(kind='bar', color='lightgreen', title="Nombres d'élèves dans chaque mention",xlabel='Mention',ylabel="Nombres d'élèves",figsize=(12, 6))


**Analyse :**

En observant le graphique ci-dessus, nous pouvons voir que de nombreux élèves se classent d'abord dans la catégorie "bien", qui correspond à une moyenne générale entre 14 et 16, et ensuite, nombreux sont ceux dans la catégorie "suffisant", qui correspond à une moyenne entre 12 et 14.



---



##Nombre d'étudiants en stage chaque année

**Description**

Nous allons ensuie pouvoir étudier le nombre d'étudiants en stage chaque année de 2017 à 2022.

In [ ]:
# Compter le nombre d'occurrences de chaque mot
type_stageCount = df['annee'].value_counts()

# Trier les données en fonction de l'axe des abscisses (label x)
type_stageCount_sorted = type_stageCount.sort_index()

# Afficher les résultats sous forme de diagramme à barres
type_stageCount_sorted.plot(kind='bar', color='skyblue', title="Nombre d'étudiants en stage chaque année de 2017 à 2022",xlabel='Année',ylabel="Nombre d'étudiant en stage",figsize=(12, 6))

**Analyse :**

Le graphique montre une augmentation initiale suivie d'une stabilisation du nombre d'étudiants effectuant des stages, avec une légère perturbation probablement due à la pandémie. Il sera important de suivre ces tendances dans les années à venir pour une analyse plus approfondie.



---



##Temps de stage

**Description**

Ici, nous cherchons à observer le temps de stage des étudiants.

In [ ]:
# Calculer la durée de chaque stage (en jours)
date['Durée de stage'] = (date['fin_stage'] - date['debut_stage']).dt.days

# Calculer la moyenne de la durée de stage
moyenne_stage = date['Durée de stage'].mean()

# Créer un graphique de dispersion
plt.figure(figsize=(10, 6))
plt.scatter(date['Durée de stage'], date['Durée de stage'], color='blue', alpha=0.5)
plt.axhline(y=moyenne_stage, color='red', linestyle='--', label='Moyenne de la durée de stage')  # Ajouter une ligne pour la moyenne
plt.title('Durée de stage en fonction de la durée de stage')
plt.xlabel('Durée de stage (jours)')
plt.ylabel('Durée de stage (jours)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

**Analyse :**

En observant le graphique de dispersion représentant la durée de stage, nous constatons que la majorité des points sont regroupés autour de la moyenne de la durée de stage. Cela suggère une certaine cohérence dans la durée des stages parmi les étudiants inclus dans l'ensemble de données. Bien que certains points puissent être situés aux extrémités du graphique, indiquant des durées de stage plus courtes ou plus longues, ils semblent être des cas exceptionnels par rapport à la tendance générale. Cette cohérence autour de la moyenne suggère une certaine stabilité ou régularité dans la durée des stages pour la population étudiée.



---



##Moyenne_Ecart-type_Variance

**Description**

Suite au dernier graphique effectué, nous allons pouvoir tirer quelques informations et les étudier, comme l'écart type des moyennes.

In [ ]:
# Calculer la moyenne des moyennes
moyenne_moyennes = df['moyenne'].mean()

# Calculer l'écart type des moyennes
ecart_type_moyennes = df['moyenne'].std()

# Calculer la variance des moyennes
variance_moyennes = df['moyenne'].var()

# Afficher les résultats
print("Moyenne des moyennes :", moyenne_moyennes)
print("Écart type des moyennes :", ecart_type_moyennes)
print("Variance des moyennes :", variance_moyennes)

**Analyse :**

À partir de ces données, voici quelques conclusions que l'on peut tirer :

Moyenne des moyennes : La moyenne générale des étudiants est d'environ 14,1.

Écart type des moyennes : L'écart type de 1,735 suggère que les moyennes des étudiants sont dispersées autour de la moyenne générale. Plus l'écart type est élevé, plus les notes des étudiants varient par rapport à la moyenne. Dans ce cas, une valeur de 1,735 indique une certaine dispersion, mais pas extrême.

Variance des moyennes : La variance mesure la dispersion des valeurs par rapport à la moyenne. Avec une variance de 3,011, cela confirme que les moyennes des étudiants varient autour de la moyenne générale.

En conclusion, les données indiquent que les étudiants ont en moyenne des résultats assez homogènes autour de 14 sur 20, avec une certaine dispersion dans les performances individuelles.



---



##Moyenne en fonction du diplôme

**Description**

Nous allons ici étudier plusieurs aspects : tout d'abord, la différence entre les moyennes de stage et de semestre entre les différents types de bac, puis déterminer si le type de bac d'un étudiant a un effet sur ses notes.

In [ ]:
# Grouper les données par diplôme (bac) et calculer la moyenne de la colonne moyenne_Notes_Semestre
moyenne_par_diplome_semestre = df.groupby('bac')['moyenne_Notes_Semestre'].mean()

# Grouper les données par diplôme (bac) et calculer la moyenne de la colonne moyenne_Notes_Stage
moyenne_par_diplome_stage = df.groupby('bac')['moyenne_Notes_Stage'].mean()

# Fusionner les deux résultats dans un seul DataFrame
moyennes = pd.concat([moyenne_par_diplome_semestre, moyenne_par_diplome_stage], axis=1)
moyennes.columns = ['Semestre', 'Stage']

# Afficher ensuite la comparaison sous forme de diagramme en bâtons
moyennes.plot(kind='bar', color=['green', 'blue'], title='Moyenne de la note semestrielle et de stage par diplôme', ylabel='Moyenne', xlabel='Bac')

# Créer une table de contingence
contingency_table = pd.crosstab(df['bac'], df['appréciation'])

# Appliquer le test du chi carré
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Afficher les résultats
print("Table de contingence :")
print("\n", contingency_table)
print("\nChi carré : ", chi2)
print("\np-value : ", p)

**Analyse :**


Sur le graphique, nous pouvons observer que les moyennes des élèves du bac S sont toujours supérieures à celles des élèves du bac techno. On remarque également que les moyennes obtenues via les notes de stages sont supérieures à celles obtenues sur le semestre. Nous pouvons donc envisager la possibilité que les notes de stage soient plus faciles à obtenir.


**Analyse de la p-value :**

La p-value obtenue est de 0.0324. Ce qui est inférieur au seuil de signification standard de 0.05.


**Interprétation de la p-value :**

Puisque la p-value est inférieure à 0.05, nous rejetons l'hypothèse nulle d'indépendance.
Cela signifie qu'il y a une association significative entre le type de bac et l'appréciation des moyennes des étudiants.


**Compréhension des résultats :**

Les résultats montrent qu'il existe une différence significative dans la répartition des appréciations en fonction du type de bac suivi par les étudiants.


**Implications pratiques**


Pour les élèves de Bac S :

La majorité des élèves se trouvent dans les catégories "Bien" et "Suffisant". Il est intéressant de noter qu'un nombre important d'étudiants a des moyennes dans ces intervalles.


Pour les élèves de Bac Techno :

Bien qu'il y ait aussi une forte concentration dans les catégories "Suffisant" et "Bien", la distribution est différente de celle des élèves de Bac S-Sciences, avec un nombre relativement élevé dans la catégorie "Insuffisant".


Pour les élèves de Bac étranger :

Comme le nombre d'étudiants est trop faible nous ne tiendrons pas contre de cette catégorie.


**Recommandations**


Interventions ciblées :

Pour les élèves de Bac Techno, il pourrait être bénéfique de renforcer les méthodes d'enseignement et de soutien pour aider à élever les notes moyennes.


Suivi et support individualisé :

Offrir un soutien personnalisé aux étudiants des catégories inférieures pour les aider à améliorer leurs performances.


**Résumé :**

Les résultats du test du chi carré indiquent qu'il existe une association significative entre le type de bac et les appréciations des moyennes des étudiants. Les différences observées suggèrent que les types de baccalauréat influencent les performances académiques des élèves.



---



##Evolution des moyennes entre le s3 et le s4

**Description**

L'objectif ici est de voir comment évolue les notes du semestre 3 au semestre 4.

In [ ]:

# Calculer les moyennes des notes du S3 et du S4
moyenne_note_s3 = df['note_s3'].mean()
moyenne_note_s4 = df['note_s4'].mean()

# Créer un DataFrame pour les moyennes
moyennes_notes = pd.DataFrame({'S3': moyenne_note_s3, 'S4': moyenne_note_s4}, index=['Moyenne'])

# Afficher les résultats sous forme de graphique à barres
ax = moyennes_notes.plot(kind='bar', figsize=(10, 6), title='Comparaison des moyennes des notes du S3 et du S4', xlabel='Semestre', ylabel='Moyenne des notes', width=0.2)


**Conclusion :**

L'augmentation de la moyenne des étudiants de 12.5 au semestre 3 à 13.6 au semestre 4 est un indicateur positif de l'amélioration des performances académiques. Cette tendance pourrait être due à une meilleure adaptation des étudiants au programme, à des méthodes d'étude améliorées, ou à un contenu de cours mieux compris. Comprendre ces facteurs peut aider à mettre en place des stratégies pour soutenir les étudiants de manière plus efficace dès le début de l'année académique.










---



##Absences et notes

In [ ]:
# Créer un graphique de dispersion
plt.figure(figsize=(10, 6))
plt.scatter(df['nb_anj_s4'], df['moyenne'], color='blue', alpha=0.5)
plt.title('Nombre d\'absences au semestre 4 par rapport à la moyenne')
plt.xlabel('Nombre d\'absences au semestre 4')
plt.ylabel('Moyenne')
plt.grid(True)
plt.tight_layout()
plt.show()

**Conclusion**

On observe sur ce graphique que durant le semestre 4, en moyenne un élève cotissant un plus grand nombre d'absences, avait une moins bonne moyenne.



---



#Tableau des données analysées

In [ ]:
donnee_univariée = {
    'Variable': ['Nominale', 'Ordinale', 'Discrète', 'continue'],
    'Donnée': ["Domaine de stage", "Appréciation","Etudiants en stage chaque annéee","Temps stage"]
}

donnee_bivariée = {
    'Variable': ['2 qualitatives', '1 de chaque', '2 quantitatives'],
    'Donnée': ["Moyenne en fonction du diplôme", "Evolution s3,s4", "Absence et note"]
}

tableau_univariée = pd.DataFrame(donnee_univariée)
tableau_bivariée = pd.DataFrame(donnee_bivariée)

print(tableau_univariée)
print("\n\n",tableau_bivariée)




---



#Conclusion

En conclusion, l'analyse approfondie des données sur les stages de deuxième année de DUT entre 2017 et 2022 permet de comprendre certaines corrélations entre les notes et les étudiants.

En examinant les données sur le nombre d'étudiants par domaine de stage, les appréciations, les évolutions annuelles, la durée des stages, les moyennes et leur relation avec le diplôme, ainsi que l'impact des absences sur les performances, nous avons pu dégager des informations riches et variées.

Ces résultats offrent des pistes de réflexion pour améliorer l'efficacité des programmes de stages, renforcer l'insertion professionnelle des étudiants, et favoriser un partenariat fructueux entre les établissements d'enseignement et les entreprises.

En continuant à exploiter ces données de manière stratégique, il est possible de prendre des décisions éclairées qui contribueront à former une main-d'œuvre qualifiée et adaptable aux défis du marché du travail contemporain.